In [ ]:
# import necessary libraries
import pandas as pd
import os

In [ ]:
## Just a check in the code that corresponding timeline and match data files were downloaded for same matches and that there were no mismatches!

matchDataDirectory = '../matchData'
matchTimelineDirectory = '../matchTimeline'

mdfiles = os.listdir(matchDataDirectory)
mlfiles = os.listdir(matchTimelineDirectory)

mdjsonfiles = [file for file in mdfiles if file.endswith('.json')]
mljsonfiles = [file for file in mlfiles if file.endswith('.json')]

print(len(mdjsonfiles))
print(len(mljsonfiles))

mdfilesset = set(mdjsonfiles)
mlfilesset = set(mljsonfiles)

if mdfilesset == mlfilesset:
    print("Corresponding match data and timeline files exists")
else:
    print("Missing match data or time line files")


In [ ]:
## There were some match files for which the API returned match data with content as null. So this is for identifying such files. 
## Correspondingly time lines should also be deleted! Deletion was done manually from terminal
## Cross-check inidividually for matchTimeline folder as well

# Define the directory and the file extension/type
directory = '../matchTimeline'
file_type = '.json'  # Change this to your desired file type

# Initialize an empty list to store the valid files
null_files = []

# Iterate through each file in the directory
for file_name in os.listdir(directory):
    # Construct the full file path
    file_path = os.path.join(directory, file_name)

    # Check if the file is of the desired type
    if file_name.endswith(file_type):
        # Open and read the content of the file
        with open(file_path, 'r') as file:
            content = file.read().strip()  # Remove any surrounding whitespace

        # Check if the content is not 'null'
        if content == "null":
            # Add the file to the valid_files list
            null_files.append(file_name)

# Output the list of null files
print("Files with null content:")
print(null_files)


In [ ]:
# read the two files containing match data and summoner's rank
matchDF = pd.read_csv('../aggregateMatchAndTimeline.csv')
print(matchDF.columns)

summonerRankDF = pd.read_csv('../summonerRanks.csv')
print(summonerRankDF.columns)

In [ ]:
matchWithSummonerRankDF = matchDF.merge(summonerRankDF, on='summonerId', how='right')
matchWithSummonerRankDF.head(20)

In [ ]:
matchPlayerCounts = matchWithSummonerRankDF.groupby(['gameId'])['summonerId'].count().reset_index()
matchPlayerCounts.rename(columns={'summonerId': 'playerCounts'}, inplace=True)
print(matchPlayerCounts.head())


matchWithSummonerRankDF = matchWithSummonerRankDF.merge(matchPlayerCounts, on='gameId', how='inner')

valid_matches = matchWithSummonerRankDF[matchWithSummonerRankDF['playerCounts'] == 10]

In [ ]:
# calculation of numerical rank of each individual player and the average rank in the play

# join the tier and rank to yield a combined value
valid_matches['tierRank'] = valid_matches['tier'] + '-' + valid_matches['rank']

rankedDict = {
    'NA':0,
    'IRON-IV':1,
    'IRON-III':2,
    'IRON-II':3,
    'IRON-I':4,
    'BRONZE-IV':5,
    'BRONZE-III':6,
    'BRONZE-II':7,
    'BRONZE-I':8,
    'SILVER-IV':9,
    'SILVER-III':10,
    'SILVER-II':11,
    'SILVER-I':12,
    'GOLD-IV':13,
    'GOLD-III':14,
    'GOLD-II':15,
    'GOLD-I':16,
    'PLATINUM-IV':17,
    'PLATINUM-III':18,
    'PLATINUM-II':19,
    'PLATINUM-I':20,
    'EMERALD-IV':21,
    'EMERALD-III':22,
    'EMERALD-II':23,
    'EMERALD-I':24,
    'DIAMOND-IV':25,
    'DIAMOND-III':26,
    'DIAMOND-II':27,
    'DIAMOND-I':28,
    'MASTER-I':29,
    'GRANDMASTER-I':30,
    'CHALLENGER-I':31
}

def calculateNumericalRank(row):
    return rankedDict[row['tierRank']]

valid_matches['numericalRank'] = valid_matches.apply(calculateNumericalRank, axis=1)

averageRankDF = valid_matches.groupby(['gameId'])['numericalRank'].mean().reset_index()
averageRankDF.rename(columns={'numericalRank': 'avgrank'}, inplace=True)

valid_matches = valid_matches.merge(averageRankDF, on='gameId', how='inner')

print(valid_matches.head())

valid_matches.sort_values(by=['gameId'], inplace=True)

In [ ]:
# export to csv
valid_matches.to_csv('../finalDataset.csv', index=False)

In [ ]:
valid_matches[['participantId','participantsAssisted','towerKillsAssisted','monsterKillsAssisted','participantsAssistedWithPressure']].head(5)

In [ ]:
list(set(valid_matches['teamId'].tolist()))

In [ ]:
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt

# Assuming your dataframe is named 'df' with columns 'to', 'from', and 'unit'
# If not, replace 'df' with your actual dataframe name
df = pd.read_csv('../data_building/team/assists.csv') ## weight file
df = df[df['gid'] == 'EUW1_6895619260']

df = df[df['weight'] != 0]
# Create a directed graph
G = nx.DiGraph()

# Add edges to the graph
for _, row in df.iterrows():
    G.add_edge(row['frm'], row['to_player'], weight=row['weight'])

# Set up the plot
plt.figure(figsize=(12, 8))

# Create a layout for the nodes
pos = nx.spring_layout(G)

# Draw the nodes
nx.draw_networkx_nodes(G, pos, node_size=800)

# Draw the edges
edges = nx.draw_networkx_edges(
    G,
    pos,
    width=2,
    edge_color='gray',
    arrowsize=12
)

# Add edge labels (weights)
edge_labels = nx.get_edge_attributes(G, 'weight')
nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels)

# Add node labels
nx.draw_networkx_labels(G, pos)

# Remove axis
plt.axis('off')

# Show the plot
plt.tight_layout()
plt.show()